In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

plt.style.use('seaborn')

In [2]:
df = pd.read_csv('../Dataset/India.csv')
df.head(n=5)

,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,...,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,population,population_density,median_age,aged_65_older,aged_70_older,life_expectancy
0,2020-01-30,1.0,1.0,NaN,NaN,NaN,NaN,0.001,0.001,NaN,...,NaN,NaN,NaN,NaN,1.380004e+09,450.419,28.2,5.989,3.414,69.66
1,2020-01-31,1.0,0.0,NaN,NaN,NaN,NaN,0.001,0.000,NaN,...,NaN,NaN,NaN,NaN,1.380004e+09,450.419,28.2,5.989,3.414,69.66
2,2020-02-01,1.0,0.0,NaN,NaN,NaN,NaN,0.001,0.000,NaN,...,NaN,NaN,NaN,NaN,1.380004e+09,450.419,28.2,5.989,3.414,69.66
3,2020-02-02,2.0,1.0,NaN,NaN,NaN,NaN,0.001,0.001,NaN,...,NaN,NaN,NaN,NaN,1.380004e+09,450.419,28.2,5.989,3.414,69.66
4,2020-02-03,3.0,1.0,NaN,NaN,NaN,NaN,0.002,0.001,NaN,...,NaN,NaN,NaN,NaN,1.380004e+09,450.419,28.2,5.989,3.414,69.66


In [3]:
# Fill Nan Values for new_cases smoothed
df.new_cases = df.new_cases.fillna(0.0)
df.new_cases_smoothed = df.new_cases_smoothed.fillna(0.0)

#Split Dataset in train test
data = np.array(df.new_cases).reshape(-1,1)
data_smoothed = np.array(df.new_cases_smoothed).reshape(-1,1)

train_data = data[:-14]
test_data = data[-14:]

train_data_smoothed = data_smoothed[:-14]
test_data_smoothed = data_smoothed[-14:]

train_data.shape, test_data.shape

((334, 1), (14, 1))

### Create Sequence Generator

In [4]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [6]:
train_generator = TimeseriesGenerator(train_data, train_data, length=7, batch_size=1)
test_generator = TimeseriesGenerator(test_data, test_data, length=7, batch_size=1)

train_generator_smoothed = TimeseriesGenerator(train_data_smoothed, train_data_smoothed, length=7, batch_size=7)
test_generator_smoothed = TimeseriesGenerator(test_data_smoothed, test_data_smoothed, length=7, batch_size=7)

### Build SequenceModel

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

In [8]:
def GetModel(input_shape = (7,1)):
    
    #build a Sequential model object
    model = Sequential()
    
    model.add(LSTM(32, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(LSTM(64, activation='relu'))
    model.add(LSTM(128, activation='relu'))
    #model.add(Dropout(0.25))
    model.add(Dense(64, activation))
    model.add(Dense(1))
    
    adam = Adam(lr=1e-2)
    model.compile(optimizer=adam, loss='mean_squared_error')
    
    return model